# It could be cake <div style="text-align: right; float: right"> IANNWTF - Final Task </div>

##### The usual imports

In [1]:
import numpy as np
import os, time
import tensorflow as tf
import json

C:\Users\nipsh\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


##### Recipe data script

In [11]:
# %load recipes.py
import pymongo, json, numpy as np
from collections import Counter
from nltk.tokenize import RegexpTokenizer


class Recipes:

    def __init__(
      self,
      useMongo=True,
      config="config.json"
    ):

        with open(config) as cfg:
            self.data = json.load(cfg)
            storage = self.data['storage']
            CRAWL_FOLDER = storage['folder']
            RECIPE_COLLECTION = storage['collection']

        client = pymongo.MongoClient('mongodb://localhost:27017')
        self.db = client['iannwtf']
        self.coll = self.db[RECIPE_COLLECTION]

    def create_dictionaries(self,vocabulary_size=20000):
        agg = self.coll.aggregate([
            {"$project": {"_id": 0, "ingredients.foodId": 1, "ingredients.name": 1}},
            {"$unwind": "$ingredients"},
            {
                "$group": {
                    "_id": "$ingredients.foodId",
                    "name": {"$first": "$ingredients.name"}
                }
            }
          ], allowDiskUse=True)
    
    

        self._ing2id = {item['name'].lower(): int(item['_id']) for item in agg}
        self._ing2id['miracel whipmyass'] = 1362
        self._ing2id['NO_INGREDIENT'] = 0
        self._id2ing = dict(zip(self._ing2id.values(), self._ing2id.keys()))

        self._word2id = {}
        self._id2word = {} 
        self._unit2id = {}
        self._id2unit = {}

        ingredient_size = len(self._ing2id)
        unit_size = len(self._unit2id)

        return ingredient_size, unit_size, vocabulary_size

    def ings2ids(self, ings):
        return self._2(ings, self._ing2id, 0)
  
    def units2ids(self, units):
        return self._2(units, self._unit2id, 0)

    def words2ids(self, words):
        return self._2(words, self._word2id, 0)

    def ids2ings(self, ids):
        return self._2(ids, self._id2ing, "NO_INGREDIENT")
  
    def ids2units(self, ids):
        return self._2(ids, self._id2unit, "NO_UNIT")

    def ids2words(self, ids):
        return self._2(ids, self._id2word, "UNKNOWN")

    def _2(self, names, d, undef):
        if type(names) in [list, range]:
            return [d.get(n, undef) for n in names]
        elif type(names) == np.ndarray:
            res = np.full_like(names, "", dtype=object)
        for i in range(res.shape[0]):
            res[i] = self._2(names[i], d, undef)
        return res
        else:
            return d.get(names, undef)

    def get_ingredient_batch(self, batchsize=25, max_ingredients=15):
        count = 0
        batch = np.zeros((batchsize, max_ingredients), dtype=np.float32)
        cursor = self.coll.find({}, {"ingredients.foodId": 1, "_id": 0}, no_cursor_timeout = True)

        recipes = np.array(list(cursor))
        np.random.shuffle(recipes)

        for doc in recipes:
        _ids = [int(x['foodId']) for x in doc['ingredients']]
        if len(_ids) > max_ingredients:
            _ids = _ids[:max_ingredients]
        batch[count][:len(_ids)] = _ids[:]
        count += 1

        if count == batchsize:
            yield batch
            count = 0
            batch = np.zeros((batchsize, max_ingredients), dtype=np.float32)

SyntaxError: invalid syntax (<ipython-input-11-7057baee5768>, line 80)

##### Helper functions

In [3]:
def feed_forward_layer(x, target_size, activation_function = None):
    """
    Courtesy of Lukas Braun. Creates a feed forward layer from input x onto size target size.
    """
    print("Forward-Layer:" + str(x.shape))
    
    fan_in = int(x.shape[-1])
    
    if activation_function == tf.nn.relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    weights = tf.get_variable("weights", [x.shape[1], target_size], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [target_size], tf.float32, var_init)
    
    activation = tf.matmul(x, weights) + biases
    
    return activation_function(activation) if callable(activation_function) else activation

def get_ings(embeddings, ingredients):
    '''
    This is used to find the ingredient id to the generated embedding of the lstm.
    We use the cosine similarity to find the closest embedding.
    
    Parameters:
    -----------
    embeddings: 2D Tensor
        the weight matrix with all ingredients representations shape: [ingredient_size, embedding_size]
    ingredients: 3D Tensor
        a matrix of ingedient vectors of who we want to find the nearest neigbhours
        
    Returns:
    --------
    closest_ings: 2D Tensor
        shape [batch_size, max_ingredients]. For each recipe in the batch, it now holds the ingredient ids
      
    '''''
    # we need to normalise for cosine similarity to work
    normed_embedding = tf.nn.l2_normalize(embeddings, dim=1)   

    # unstacking and restacking since matrix multiplications only work on the same rank
    closest_ings = []
    for batch in tf.unstack(ingredients, axis=1):
        batch = tf.nn.l2_normalize(batch, dim=1)
        closest_ings.append(tf.argmax(tf.matmul(batch, tf.transpose(normed_embedding, [1, 0]))))

    closest_ings = tf.stack(closest_ings, axis=1)
    
    return closest_ings

def discriminator_convolution(x, window_size) : 
    """
    Convolutes over the recipe in its embedding form, to extract features.
    Which are used to discriminate between the original recipes and the generated ones.
    
    Params:
    -------
    x: 3D Tensor
        The tensor holding the original recipes and the generated ones with their embeddings
    window_size:  int
        the window size to use in the convolution
        
    Returns:
    --------
    convolution: 2d tensor
        the result of max pooling after the convoliution for feature extraction
    """
    
    x = tf.reshape(x,[2*batch_size, embedding_size, max_ingredients, 1])
    fan_in = 1
    for dim in x.shape[1:]: fan_in * dim.value
    
    var_init = tf.random_normal_initializer(stddev=fan_in**1/2)
    kernel = tf.get_variable("kernel", [embedding_size, window_size, 1,1], tf.float32, var_init)
    
    y = tf.nn.conv2d(x, kernel, strides = [1, 1, 1, 1], padding = "VALID")    
    new_length = y.get_shape()[2]
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [1,1, new_length, 1], initializer = var_init)
    
    y = y + biases
    
    return tf.squeeze(tf.nn.max_pool(y, [1,1, new_length, 1], strides = [1, 1, 1, 1], padding = "VALID"))

def generator_similarity_loss(batch):
    """
    This loss is introduced to make the network choose more diverse ingredients, instead of repeating
    ingredients and using close ingredients
    
    Params:
    -------
    batch: 3d Tensor
        The generated recipe batch
        
    Returns:
    --------
        The mean cosine similarity for each recipe to itself
    """
    def iterate(recipe):
        tn = tf.nn.l2_normalize(recipe, dim=1)
        similarity = tf.matmul(tn, tf.transpose(tn, [1,0]))
        sim = tf.reduce_mean(similarity)
        return sim
    
    all_sims = tf.map_fn(iterate, batch)
        
    
    return tf.stack(all_sims)

##### Loading recipe data, defining hyperparameters

In [15]:
config = "config.json"
recipes = Recipes(config)

with open(config,"r",encoding="utf-8") as cfg:
    settings = json.load(cfg)["network"]

hyperparams = settings["hyperparameters"]
directories = settings["directories"]

epochs = 6
learning_rate = 0.0004
beta1 = 0.5
batch_size = 50
lambda_ing = 0.4 # coefficient to use for the custom loss

# the pretrained weights
embedding_weights = directories["embedding_weights"]

# for tensorboard
weight_dir = directories["weight_dir"]
summary_dir = directories["summary_dir"]
     
## Hyperparameters
embedding_size = hyperparams["embedding_size"]
# size of the random input vector
z_size = hyperparams["z_size"]
max_ingredients = hyperparams["max_ingredients"]
lstm_memory_size = embedding_size
dropout_rate = hyperparams["dropout_rate"]
optimizer = tf.train.AdamOptimizer

# and create the dictionaries to look up the ingredients.
# unit_size & vocab size are unused and were implemented for further extension of the network
ingredient_size, unit_size, vocabulary_size = recipes.create_dictionaries()

print("""embedding size : {},
input size : {},
max ingredients : {},
lstm state size : {},
dropoutrate : {},
ingredient size : {}"""\
      .format(embedding_size, z_size, max_ingredients, lstm_memory_size, dropout_rate, ingredient_size))

embedding size : 64,
input size : 50,
max ingredients : 10,
lstm state size : 64,
dropoutrate : 0.85,
ingredient size : 3601


##### Defining the graph

In [8]:
# Not resetting causes conflicts in jupyter as the previous executed graph is still active in memory
tf.reset_default_graph()


###### The generator network. ######      
with tf.variable_scope("generator"):

    ##### placeholders  #####
    # for input vector, and lstm state
    with tf.variable_scope("inputs"):
        input_vec = tf.placeholder(tf.float32,[batch_size,z_size],name="input_vec")
        
        cell_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size],name="cell_state")
        hidden_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size],name="hidden_state")
    
    
    ##### embedding #####
    # it is not trainable as it was pre trained to a sufficient degree
    with tf.variable_scope("embedding"):
        init = tf.random_uniform_initializer(-1.0,1.0)
        embeddings = tf.get_variable("embed",
                                     [ingredient_size, embedding_size],
                                     initializer=init,
                                     trainable=False
                                    )
  
        
    ##### RNN #####
    # nothing special about it. We repeatedly feed the input vector for each ingredient to be produced
    with tf.variable_scope("rnn"):
        cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_memory_size)
        cell= tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout_rate)
        zero_state = cell.zero_state(batch_size, tf.float32)
             
        init_state = tf.nn.rnn_cell.LSTMStateTuple(c = cell_state, h = hidden_state)
        inputs = [input_vec for i in range(max_ingredients)]
        
        lstm_out, lstm_state = tf.nn.static_rnn(cell, inputs, initial_state=init_state)
        
        # has shape [max_ingredients, batch_size, embedding], but we need batch_size major
        outputs = tf.transpose(lstm_out, [1,0,2])

    # get trainable variables for generator
    train_gen = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)


##### The discriminator network #####
with tf.variable_scope("discriminator"):
    
    ##### placeholder for originals & batch creation #####
    with tf.variable_scope("input"):
        orig_recipes_placeholder = tf.placeholder(tf.int64,
                                                  [batch_size,max_ingredients],
                                                  name="orig_recipes_placeholder"
                                                 )
        orig_recipes = tf.nn.embedding_lookup(embeddings, orig_recipes_placeholder)

        # create input batch for discriminator
        batch = tf.concat([outputs, orig_recipes],axis=0)
    
    ##### CNN #####
    with tf.variable_scope("convolutions"):
        convs = []
        for i in range(2,max_ingredients+1):
            with tf.variable_scope("conv"+str(i)):
                conv = discriminator_convolution(batch, i)
                convs.append(conv)
        
        conv_outputs = tf.stack(convs, axis=1)
     
    ##### Single Node reduction #####
    with tf.variable_scope("readout"):
        logits = feed_forward_layer(conv_outputs,1,None)
                                    
    # get trainable variables for discriminator
    train_dis = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

##### Evaluation and training of the network #####
with tf.variable_scope("evaluation"):
    
    # to look at what glorious ingredient compilations it produced
    readable_outputs = get_ings(embeddings, outputs)
    
    # to compute cross entropy for generator.
    # consists of only "1" because this is the discriminator label for "real"
    gen_labels = tf.ones((batch_size,1))
    
    # only the first half of the output from the discriminator concerns the pictures
    # produced by the generator
    # so only get first half of logits (which equals the batch_size)
    gen_logits = logits[:batch_size]
    
    # for discriminator cross entropy.
    # first half of input are fake images ("0"), second half real ones ("1")
    dis_labels = tf.concat((tf.zeros((batch_size,1)),tf.ones((batch_size,1))),axis=0)
    dis_logits = logits

    # calculating the loss for generator and discriminator
    gen_loss_mult_all = generator_similarity_loss(outputs)
    gen_loss_multing = tf.reduce_mean(gen_loss_mult_all) * lambda_ing
    gen_loss_ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=gen_labels, logits=gen_logits)
    gen_loss = gen_loss_ce + gen_loss_multing
    
    dis_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=dis_labels, logits=dis_logits)
    
    # initialize optimizer
    optimizer  = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
    
    # define training steps with respective variable lists
    global_step = tf.get_variable("global_step",[],tf.int32,trainable=False)
    gen_step = optimizer.minimize(gen_loss, var_list=train_gen, global_step=global_step)
    dis_step = optimizer.minimize(dis_loss, var_list=train_dis, global_step=global_step)
    
    # tensorboard things
    tf.summary.scalar("generator_loss", tf.reduce_mean(gen_loss))
    tf.summary.scalar("discriminator_loss", tf.reduce_mean(dis_loss))
    summaries = tf.summary.merge_all()

Tensor("generator/rnn/transpose:0", shape=(50, 10, 64), dtype=float32)
Instructions for updating:
dim is deprecated, use axis instead
New shape after convolution:(100, 1, 9, 1)
New shape after convolution:(100, 1, 8, 1)
New shape after convolution:(100, 1, 7, 1)
New shape after convolution:(100, 1, 6, 1)
New shape after convolution:(100, 1, 5, 1)
New shape after convolution:(100, 1, 4, 1)
New shape after convolution:(100, 1, 3, 1)
New shape after convolution:(100, 1, 2, 1)
New shape after convolution:(100, 1, 1, 1)
(100, 9)
Forward-Layer:(100, 9)


##### This is where we train our network

In [14]:
with tf.Session() as session:
    
    ## initialising directories if they do not exist
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    if not os.path.exists(summary_dir):
        os.makedirs(summary_dir)
        
    # To save what we do. Like a diary, but for ann's
    train_saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter(summary_dir, session.graph)
    
    ## Try to load trained data and initialise variables if there is none
    ckpt = tf.train.latest_checkpoint(weight_dir)
    if ckpt is None:
        print("No checkpoint found. Initialising variables.")
        session.run(tf.global_variables_initializer())
    else:
        train_saver.restore(session, ckpt)    
    
    # load the pre trained embedding. If it does not exist, this is not gonna work.
    embed_saver = tf.train.Saver(var_list={'ingredient_embedding': embeddings})
    if os.path.isdir(embedding_weights):
        embed_ckpt = tf.train.latest_checkpoint(embedding_weights)
        if embed_ckpt is not None:
            embed_saver.restore(session, embed_ckpt)
        else:
            raise Exception("Embedding weights not found")
    else:
        raise Exception("Embedding weights folder not found")
    
    
    ## Train all the epochs and save the summaries for each timestep
    for epoch in range(1, epochs+1):
        
        print("Starting epoch %d..." %epoch)
        
        # get batch of original recipes #mnist.get_batch(batch_size)
        real_data = recipes.get_ingredient_batch(batch_size, max_ingredients)
        t = time.time()
        
        counter = 0
        
        for data in real_data:
            # lstm initial state will be 0
            _state = session.run(zero_state)
            
            ## for each batch we need a new set of random vectors
            z_vec = np.random.uniform(-1, 1, (batch_size,z_size))            
            
            # feed data into placeholders
            _feed_dict = {input_vec: z_vec, 
                         orig_recipes_placeholder: data,
                         cell_state: _state.c,
                         hidden_state: _state.h,}
            
            # let's train our network!
            _gStep, _dStep, _summ, _step, _fr = session.run([
                                                    gen_step, dis_step,
                                                    summaries, global_step,
                                                    readable_outputs
                                                ],
                                                feed_dict=_feed_dict)
            
            train_writer.add_summary(_summ, _step)
            
            # create the ingredient names. numbers are hard to read
            _fr = recipes.ids2ings(_fr)
            # output every 100 steps            
            if counter % 100 == 0 :
                for rec in np.random.choice(range(batch_size),5):
                    print("|| ",end='')
                    for i in _fr[rec]:
                        print(i," + ",end='')
                    print(" ||")
                print("------------------------------------------")
            
            counter = counter + 1
            
        t = time.time() - t
        est = t * (epochs - epoch)
        
        ## This is just the time formatting. Move along.
        m, s = divmod(t, 60)
        e_m, e_s = divmod(est, 60)
        
        print(("Finished epoch in {0: .0f}m{1: .2f}s. "+\
              "Estimated remaining time: {2: .0f}m{3: .2f}s").format(m,s,e_m,e_s))
        
        # and save the weights
        train_saver.save(session, weight_dir, global_step=_step)

INFO:tensorflow:Restoring parameters from weights/-50912
INFO:tensorflow:Restoring parameters from embedding_weights/model.ckpt
Starting epoch 1...
|| milch  + milch  + milch  + honig  + wodka  + milch  + gemüsebrühe  + galbani mozzarella  + guaranapulver  + milch  +  || loss: 0.8289738
|| zitronensaft  + ananas  + ingwerpulver  + zitronensaft  + zitronensaft  + schafskäse  + möhre(n)  + galbani mozzarella  + galbani mozzarella  + milch  +  || loss: 0.8316986
|| zitronensaft  + löwensenf (medium)  + blue curaçao  + lauchzwiebel(n)  + limette(n)  + tomatensaft  + chilischote(n)  + paniermehl  + pitu (aguardente de cana)  + knoblauch  +  || loss: 0.89181554
|| zitronensaft  + ananas  + ingwerpulver  + ingwerpulver  + zitronensaft  + löwensenf (medium)  + chilischote(n)  + paniermehl  + pitu (aguardente de cana)  + schafskäse  +  || loss: 0.7384643
|| zitronensaft  + ananas  + löwensenf (medium)  + chilischote(n)  + limette(n)  + möhre(n)  + ei(er)  + ingwerpulver  + zwiebel(n)  + zwiebel

|| ananas  + schafskäse  + NO_INGREDIENT  + mehl  + paprikapulver  + oliven  + galbani mozzarella  + champignons  + salatgurke(n)  + salz  +  || loss: 0.83844066
|| pitu (aguardente de cana)  + limettensaft  + gin  + oliven  + blue curaçao  + pitu (aguardente de cana)  + ananas  + lollo rosso  + ingwerpulver  + zwiebel(n)  +  || loss: 0.82664216
|| ananas  + salatgurke(n)  + appel delikatess mayonnaise  + hackfleisch  + mehl  + oliven  + salz  + mehl  + frühlingszwiebel(n)  + oliven  +  || loss: 0.8143807
|| ananas  + ananas  + mehl  + ei(er)  + paprikapulver  + ananas  + galbani mozzarella  + ananas  + oliven  + galbani mozzarella  +  || loss: 0.83840615
|| limettensaft  + lauchzwiebel(n)  + milch  + limettensaft  + cognac  + ingwerpulver  + cognac  + limettensaft  + ananassaft  + ingwerpulver  +  || loss: 0.8039466
------------------------------------------
|| NO_INGREDIENT  + cognac  + zitronensaft  + limette(n)  + zucker, braun  + NO_INGREDIENT  + cognac  + aubergine(n)  + galbani 

|| grapefruitsaft  + salatgurke(n)  + karambolenscheibe(n)  + tomatensaft  + ananas  + zwiebel(n)  + butter  + champignons  + appel delikatess mayonnaise  + lauchzwiebel(n)  +  || loss: 0.82848287
|| cognac  + karambolenscheibe(n)  + milch  + knoblauch  + knoblauch  + milch  + karambolenscheibe(n)  + aubergine(n)  + milch  + karambolenscheibe(n)  +  || loss: 0.83053607
|| basilikum  + salatgurke(n)  + karambolenscheibe(n)  + knoblauch  + NO_INGREDIENT  + milch  + ananas  + knoblauch  + milch  + lauchzwiebel(n)  +  || loss: 0.5774985
|| cognac  + galbani mozzarella  + cognac  + galbani mozzarella  + galbani mozzarella  + galbani mozzarella  + galbani mozzarella  + cognac  + galbani mozzarella  + milch  +  || loss: 0.8735426
|| gemüsebrühe  + pitu (aguardente de cana)  + grenadine  + mehl  + zucker, braun  + champignons  + guaranapulver  + schafskäse  + honig  + basilikum  +  || loss: 0.8075625
------------------------------------------
|| tomate(n)  + friséesalat  + friséesalat  + limet

|| möhre(n)  + chilischote(n)  + guaranapulver  + guaranapulver  + schafskäse  + limette(n)  + guaranapulver  + limette(n)  + koriandergrün  + ei(er)  +  || loss: 0.8218724
|| koriandergrün  + chilischote(n)  + guaranapulver  + ananas  + limette(n)  + mehl  + guaranapulver  + tomatensaft  + pitahaya(s)  + frühlingszwiebel(n)  +  || loss: 0.64615893
|| gin  + chilischote(n)  + lauchzwiebel(n)  + guaranapulver  + NO_INGREDIENT  + basilikum  + ananas  + basilikum  + koriandergrün  + ei(er)  +  || loss: 0.78847003
|| möhre(n)  + chilischote(n)  + guaranapulver  + guaranapulver  + tomatensaft  + limette(n)  + guaranapulver  + limette(n)  + tomatensaft  + ei(er)  +  || loss: 0.82824814
|| möhre(n)  + chilischote(n)  + guaranapulver  + guaranapulver  + NO_INGREDIENT  + limette(n)  + guaranapulver  + limette(n)  + koriandergrün  + ei(er)  +  || loss: 0.81301695
------------------------------------------
|| cognac  + wodka  + koriandergrün  + knoblauch  + NO_INGREDIENT  + honig  + limettensaft 

|| oliven  + knoblauch  + zwiebel(n)  + zwiebel(n)  + limette(n)  + zwiebel(n)  + ingwerpulver  + galbani mozzarella  + guaranapulver  + pitahaya(s)  +  || loss: 0.7224604
|| galbani parmesan  + ingwerpulver  + salz  + milch  + pitahaya(s)  + salatgurke(n)  + salz  + lollo rosso  + schafskäse  + zwiebel(n)  +  || loss: 0.8681734
|| gin  + tomate(n)  + tomate(n)  + aubergine(n)  + tomate(n)  + champignons  + knoblauch  + tomate(n)  + champignons  + champignons  +  || loss: 0.8523748
|| gin  + tomate(n)  + tomate(n)  + aubergine(n)  + champignons  + champignons  + tomate(n)  + champignons  + champignons  + champignons  +  || loss: 0.7242641
|| gin  + lollo rosso  + salz  + basilikum  + cognac  + basilikum  + salz  + ei(er)  + frühlingszwiebel(n)  + salatgurke(n)  +  || loss: 0.842669
------------------------------------------
|| möhre(n)  + hackfleisch  + möhre(n)  + lauchzwiebel(n)  + galbani parmesan  + grenadine  + chilischote(n)  + grenadine  + grenadine  + salatgurke(n)  +  || loss:

|| guaranapulver  + gemüsebrühe  + galbani mozzarella  + ingwerpulver  + paniermehl  + möhre(n)  + limettensaft  + ingwerpulver  + paniermehl  + karambolenscheibe(n)  +  || loss: 0.7737046
|| ananassaft  + paprikapulver  + ananassaft  + champignons  + grenadine  + koriandergrün  + friséesalat  + ananassaft  + champignons  + friséesalat  +  || loss: 0.7737388
|| limettensaft  + zucker, braun  + lollo rosso  + grapefruitsaft  + oliven  + grapefruitsaft  + pfeffer  + oliven  + appel delikatess mayonnaise  + oliven  +  || loss: 0.78857046
|| honig  + zucker, braun  + koriandergrün  + möhre(n)  + ei(er)  + grapefruitsaft  + pfeffer  + chilischote(n)  + appel delikatess mayonnaise  + schafskäse  +  || loss: 0.8248269
|| guaranapulver  + paniermehl  + galbani mozzarella  + pitahaya(s)  + paniermehl  + frühlingszwiebel(n)  + limettensaft  + ingwerpulver  + salatgurke(n)  + karambolenscheibe(n)  +  || loss: 0.76814216
------------------------------------------


KeyboardInterrupt: 